In [1]:
#noetbook utilizado para a aula sobre sistemas de recomendação

#1) Carregar os dados
#2) Pré-processamento dos dados
#3) Divisão dos dados entre treinamento e teste
#4) Criar o sistema de recomendação através do ALS
#5) Realizar previsões
#6) Testar o modelo

In [3]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [4]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("recomendacao").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [5]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/Mall_Customers.csv,Mall_Customers.csv,4286
dbfs:/FileStore/tables/adult_data.csv,adult_data.csv,5608318
dbfs:/FileStore/tables/d1995_07_01_24d0c.json,d1995_07_01_24d0c.json,7451741
dbfs:/FileStore/tables/d1995_07_02_c3f44.json,d1995_07_02_c3f44.json,9572086
dbfs:/FileStore/tables/d1995_07_03_f99af.json,d1995_07_03_f99af.json,14125303
dbfs:/FileStore/tables/d1995_07_04_c5a7f.json,d1995_07_04_c5a7f.json,12030333
dbfs:/FileStore/tables/d1995_07_05_0f261.json,d1995_07_05_0f261.json,14662194
dbfs:/FileStore/tables/d1995_07_06_20619.json,d1995_07_06_20619.json,15557682
dbfs:/FileStore/tables/d1995_07_07_2dd8d.json,d1995_07_07_2dd8d.json,15279295
dbfs:/FileStore/tables/d1995_07_08_83302.json,d1995_07_08_83302.json,7033852


In [6]:
diretorioRecomendacao="/FileStore/tables/u.data"  #diretório que contém o arquivo a ser utilizado

#1) Carregando o arquivo.

In [8]:
#lendo arquivos armazenados através da função genérica
rdd_movies = spark.sparkContext.textFile(diretorioRecomendacao)

In [9]:
rdd_movies.take(10)  #user id | item id | rating | timestamp

Out[7]: ['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [10]:
#definindo as bibliotecas a serem utilizadas
from pyspark.mllib.recommendation import ALS, Rating  #MLlib utilizada para implementar os algoritmos ALS e Rating

#2) Pré-processamento dos dados
#3) Separando os dados para treinamento e teste

In [12]:
#dividindo os dados entre treinamento e teste
(trainRatings, testRatings) = rdd_movies.randomSplit([0.7, 0.3])

In [13]:
trainRatings.take(5)

Out[10]: ['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [14]:
testRatings.first()  #print da primeira linha do RDD

Out[11]: '298\t474\t4\t884182806'

In [15]:
trainingData = trainRatings.map(lambda l: l.split('\t')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))  #aplicando a função Rating

In [16]:
trainingData.first()  #print do RDD criado através da função Rating

Out[15]: Rating(user=196, product=242, rating=3.0)

In [17]:
#mesmo procedimento para os dados de teste
testData = testRatings.map(lambda l: l.split('\t')).map(lambda l: (int(l[0]), int(l[1])))

In [18]:
testData.first()  #id do usuário | id do filme

Out[17]: (298, 474)

#4) Construindo o modelo ALS

In [20]:
#definindo as variáveis do modelo
rank = 10  # número de fatores latentes do modelo R-> P (usuários) * Q (itens) => R_mxn = P_mxrank * Q_rankxn  (em que m = número de usuários e n = número de itens)
numIterations = 50 #número de iterações realizadas pelo modelo
model = ALS.train(trainingData, rank, numIterations) # treina o modelo

#5) Previsão do modelo

In [22]:
model.predict(253, 465)  # entrada (usuário,produto)

Out[19]: 4.287589950673622

In [23]:
previsao = model.predictAll(testData)  #previsão para todos os dados de teste
previsao.first()

Out[20]: Rating(user=264, product=320, rating=6.248419678505375)

In [24]:
previsao = previsao.map(lambda l: ((l[0], l[1]), l[2])) #mapea para a exibição
previsao.take(5)

Out[21]: [((264, 320), 6.248419678505375),
 ((896, 320), 1.1423426822713156),
 ((617, 320), 0.24644247517994788),
 ((385, 320), 4.952559029673385),
 ((3, 320), 4.0792978378586735)]

In [25]:
testRating2 = testRatings.map(lambda l: l.split('\t')).map(lambda l: ((int(l[0]), int(l[1])), float(l[2]))) # mapea para exibição e utilização na análise

In [26]:
testRating2.first()

Out[23]: ((298, 474), 4.0)

In [27]:
ratingsAndPredictions = testRating2.join(previsao)
ratingsAndPredictions.take(5)

Out[24]: [((305, 451), (3.0, 1.7894887819533256)),
 ((210, 40), (3.0, 2.545046565846654)),
 ((234, 1184), (2.0, 1.5396540580167994)),
 ((20, 288), (1.0, 1.78221869310718)),
 ((90, 648), (4.0, 4.705448875260114))]

#6) Avaliação do modelo

In [29]:
MSE = ratingsAndPredictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print ("Erro médio quadrado = " + str(MSE))

Erro médio quadrado = 1.3386061862139509

In [30]:
#dado um usuário -> recomenda os top 5 produtos
model.recommendProducts(105, 5) 

Out[26]: [Rating(user=105, product=793, rating=10.443188147641196),
 Rating(user=105, product=787, rating=10.034698170580358),
 Rating(user=105, product=464, rating=7.993213890871889),
 Rating(user=105, product=548, rating=7.992998353523128),
 Rating(user=105, product=1006, rating=7.886061617503781)]

In [31]:
#dado um produto -> recomenda top 5 usuários
model.recommendUsers(1, 5)  #filme Toy Story (1995)

Out[27]: [Rating(user=258, product=1, rating=6.4252115758349815),
 Rating(user=166, product=1, rating=6.074740296540562),
 Rating(user=810, product=1, rating=6.074598264087588),
 Rating(user=688, product=1, rating=5.841285272238177),
 Rating(user=408, product=1, rating=5.8059534823107155)]

In [32]:
#mostrando o vetor de características referentes usuários (coluna - P)
model.userFeatures().take(1)[0]

Out[28]: (8,
 array('d', [0.17623691260814667, 0.3033490777015686, 0.8925564289093018, -0.09665598720312119, 1.6530667543411255, -0.09452690184116364, 0.3103662133216858, -1.7551463842391968, -0.12293795496225357, -0.20986689627170563]))

In [33]:
#mostra o vetor de características referente a um produto (linha - Q)
model.productFeatures().take(1)[0]

Out[29]: (8,
 array('d', [0.6616284847259521, 0.7762099504470825, 0.7257583737373352, -0.9608121514320374, 1.9275953769683838, -0.4532114565372467, 0.5060295462608337, -0.33511659502983093, 0.36126622557640076, 0.15067516267299652]))